# How to use the optimisation classes

This notebook provides example usage on PyBOP's optimisation classes.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP from its development branch and upgrade some dependencies:

In [1]:
%pip install --upgrade pip ipywidgets
%pip install pybop -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importing Libraries

With the environment set up, we can now import PyBOP alongside other libraries we will need:

In [2]:
import numpy as np

import pybop

## Importing Parameters

This can be completed by importing a JSON representation, such as the one in the PyBOP [examples](https://github.com/pybop-team/PyBOP/blob/develop/examples/scripts/parameters/initial_ecm_parameters.json). To import via JSON, either download the example file, or create your own and update the path below to reference the corresponding file.

In [3]:
parameter_set = pybop.ParameterSet(
    json_path="../scripts/parameters/initial_ecm_parameters.json"
)
parameter_set.import_parameters()

{'chemistry': 'ecm',
 'Initial SoC': 0.5,
 'Initial temperature [K]': 298.15,
 'Cell capacity [A.h]': 5,
 'Nominal cell capacity [A.h]': 5,
 'Ambient temperature [K]': 298.15,
 'Current function [A]': 5,
 'Upper voltage cut-off [V]': 4.2,
 'Lower voltage cut-off [V]': 3.0,
 'Cell thermal mass [J/K]': 1000,
 'Cell-jig heat transfer coefficient [W/K]': 10,
 'Jig thermal mass [J/K]': 500,
 'Jig-air heat transfer coefficient [W/K]': 10,
 'Open-circuit voltage [V]': <function pybamm.input.parameters.ecm.example_set.ocv(sto)>,
 'R0 [Ohm]': 0.001,
 'Element-1 initial overpotential [V]': 0,
 'Element-2 initial overpotential [V]': 0,
 'R1 [Ohm]': 0.0002,
 'R2 [Ohm]': 0.0003,
 'C1 [F]': 10000,
 'C2 [F]': 5000,
 'Entropic change [V/K]': 0.0004}

## Identifying the Parameters

Now that the initial parameter set is constructed, we can start the PyBOP fitting process. First, we define the model class with two RC elements.

In [4]:
model = pybop.empirical.Thevenin(
    parameter_set=parameter_set, options={"number of rc elements": 2}
)

In this example, we are going to try to fit all five parameters at once. This isn't recommend for real-life application as identifiablity is challenging to guarantee with this large a parameter space. To do this, we define the `pybop.parameters` as,

In [5]:
parameters = [
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(0.0002, 0.0001),
        bounds=[1e-4, 1e-2],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(0.0001, 0.0001),
        bounds=[1e-5, 1e-2],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(0.0001, 0.0001),
        bounds=[1e-5, 1e-2],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(10000, 2500),
        bounds=[2500, 5e4],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(10000, 2500),
        bounds=[2500, 5e4],
    ),
]

Let's create some synthetic data to identify the parameters. This data is then corrupted with a small amount of Gaussian noise to represent some additional uncertainty in the measured values. We can then form the `pybop.Dataset` from this data.

In [6]:
sigma = 0.001
t_eval = np.arange(0, 900, 2)
values = model.predict(t_eval=t_eval)
corrupt_values = values["Voltage [V]"].data + np.random.normal(0, sigma, len(t_eval))

# Form dataset
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": values["Current [A]"].data,
        "Voltage [V]": corrupt_values,
    }
)

The `FittingProblem` class provides us with a single class that holds all of the objects we need to evaluate our selected `SumSquaredError` cost function. 

In [7]:
problem = pybop.FittingProblem(model, parameters, dataset)
cost = pybop.SumSquaredError(problem)

The cost function can be interrogated manually via the `cost([params])` API. In this example, that would look like the following,

In [8]:
cost([0.001, 0.001, 0.001, 5000, 5000])

0.02585144927344025

## Interacting with the Optimisation classes

Next, we construct the optimisation class with our algorithm of choice and run it. For the first case, we show the direct optimiser interface and construct the `pybop.CMAES` method. For the sake of reducing the runtime of this example, we limit the maximum iterations to 50; however, feel free to update this value. To showcase the different methods to set optimisation options, this example sets the maximum iterations three different ways.

In [ ]:
optim_interface_one = pybop.CMAES(cost, max_iterations=50)
optim_interface_one.set_max_iterations(50)
x1, final_cost = optim_interface_one.run(max_iterations=50)

This can also be completed via the `pybop.Optimisation` interface as shown below. The method is less direct than the previous one, but provides a single class to work with across PyBOP workflows.

In [ ]:
optim_interface_two = pybop.Optimisation(cost, optimiser=pybop.CMAES, max_iterations=50)
optim_interface_two.set_max_iterations(50)
x2, final_cost = optim_interface_two.run(max_iterations=50)

Finally, we can compare the output from these methods.

In [11]:
print("Initial parameters:", cost.x0)
print("Estimated parameters x1:", x1)
print("Estimated parameters x2:", x2)

Initial parameters: [2.29696565e-04 3.53341865e-05 1.63145688e-05 1.07259649e+04
 9.73352990e+03]
Estimated parameters x1: [9.22523088e-04 1.68211333e-04 4.06342748e-04 1.07259640e+04
 9.73352980e+03]
Estimated parameters x2: [8.43095153e-04 3.51316155e-04 2.95401229e-04 1.07259654e+04
 9.73352938e+03]


## Conclusions

In this notebook, we present two optimisation interfaces for optimisation within PyBOP. Both of these interfaces are offered to cater to end-user preferences.